In [2]:
%matplotlib inline
import random
from operator import itemgetter
import numpy as np
from multiprocessing.pool import ThreadPool

from pyquil.quil import Program
from pyquil.api import QVMConnection

In [3]:
import os

API_KEY = 'nmRPAVunQl19TtQz9eMd11iiIsArtUDTaEnsSV6u'
USER_ID = '221422a5-07ab-4cb2-8d4d-2b3dae3aedc7'

PYQUIL_CONFIG = f"""
[Rigetti Forest]
url: https://api.rigetti.com/qvm
key: {API_KEY}
user_id: {USER_ID}
"""

with open(os.path.expanduser('~/.pyquil_config'), 'w') as f:
    f.write(PYQUIL_CONFIG)

In [4]:
def all_same(items):
    return all(x == items[0] for x in items)

def tester(bitstring,n=2,m=2):
   # print(np.array(bitstring))
    mySplit = np.split(np.array(bitstring),n)
    for line in mySplit:
        if not all_same(line):
            mySplit = np.transpose(mySplit)
            for col in mySplit:
                if not all_same(col):
                    return False
            return True
    return True

In [5]:
#This class can be used to optimize the structure of a fully connected network for any problem. 
class DDQCL():
    def __init__(self, init_depth=6, num_qubits=4):
        self.gate_dict={0:['X', 1], 1:['Y', 1], 2:['H', 1], 3:['Z', 1], 4:['CNOT', 2], 5:['ISWAP', 2]}
        
        self.init_depth = init_depth
        self.chrom_len = 3
        self.num_gates = len(self.gate_dict)
        self.num_qubits = num_qubits
        
        self.best_gene = []
        self.best_fitness = 0
    
    def program_constructor(self, gene):        
        '''Takes in the gene as a parameter and returns the neural network the gene codes for'''
        program_string=''
        for chrom in gene:
            gate_info = self.gate_dict[chrom[0]]
            program_string+= gate_info[0]+ ''.join([' '+str(qubit) for qubit in chrom[1:gate_info[1]+1]]) + ' \n'
        program_string+='MEASURE 0[0] \nMEASURE 1[1] \nMEASURE 2[2] \nMEASURE 3[3]'
        return program_string
    
    #crossover operation between two parent genes
    def crossover(self, gene1, gene2):
        '''Created a combined gene by splicing the two parent genes together
        and samples from it to create the child gene'''
        child = []
        maxlen = max(len(gene1), len(gene2))
        combined = []
        for k in range(maxlen):
            if k<len(gene1):
                combined.append(gene1[k])
            if k<len(gene2):
                combined.append(gene2[k])

        for chrom in combined:
            prob_sample = random.uniform(0, 1)
            if prob_sample>=0.5:
                child.append(chrom)
        if len(child)==0:
            child.append(gene1[0])
        return child
    
    def breed(self, breed_percent):
        '''Replace bottom breed_percent% of the population with children created
        by performing crossover operation on best performing candidates' genes '''
        number_replace = int(len(self.population_genes)*breed_percent)
        for k in range(number_replace):
            parents = random.sample(self.population_genes[number_replace:], 2)
            child = self.crossover(parents[0], parents[1])
            self.population_genes[k] = child

    def mutate(self, mutation_prob=0.1):
        '''Add noise to each candidate's gene in the population'''
        for i, gene in enumerate(self.population_genes[:-5]):
            for j, chrom in enumerate(gene):
                for k, genotype in enumerate(chrom):
                    prob_sample = random.uniform(0, 1)
                    if prob_sample<mutation_prob:
                        if k==0:
                            self.population_genes[i][j][k] = random.randint(0, self.num_gates-1)
                        else:
                            self.population_genes[i][j][k] = random.randint(0, self.num_qubits-1)
                            #ensure three unique qubits for operations
                            qubitset = set(self.population_genes[i][j][1:]) 
                            while len(qubitset) != self.chrom_len-1 or qubitset==set((0, 2)) or qubitset==set((0, 3)) or qubitset==set((1, 3)): 
                                self.population_genes[i][j][k] = random.randint(0, self.num_qubits-1)
                                qubitset = set(self.population_genes[i][j][1:])
                            break
    
    def evaluate_fitness(self, datinput, trials=1000):
        """
        Thin function that takes a low-level Quil program and returns the
        resulting probability distribution.
        """
        i, quil_program = datinput
        qvm = QVMConnection()
        results = qvm.run(Program(quil_program), trials=trials)
        results = list(map(tuple, results))

        observed_results = set(results)
        recall = 0
        precision = 0
        for result in sorted(observed_results):
            bitstring = ''.join(reversed(list(map(str, result))))
            failer = tester(list(map(str, result)))
            p_i = results.count(result)/len(results) +0.0001
            if failer:
                recall+=1
                precision += p_i
                
            #print(str(failer) + f'|{bitstring}> state: {p_i} [{results.count(result)}/{len(results)}]')
        recall /= 6
        f1 = (2*precision*recall)/(precision+recall+0.0001)
        #print(f1)
        self.fitnesses[i] = f1
        return f1
        
    def run(self, pop_size=50, gens=5, n_top=5):
        '''Perform the optimization using the rest of the functions defined in this class'''
        #initialize population
        self.population_genes = []
        self.fitnesses = [0]*pop_size
        
        #initialize the population with random genes
        for i in range(pop_size):
            depth = random.randint(3, self.init_depth)
            gene = []
            for j in range(depth):
                chrom = []
                chrom.append(random.randint(0, self.num_gates-1))
                for k in range(self.chrom_len-1):
                    chrom.append(random.randint(0, self.num_qubits-1))
                
                qubitset = set(chrom[1:])
                while len(qubitset) != self.chrom_len-1 or qubitset==set((0, 2)) or qubitset==set((0, 3)) or qubitset==set((1, 3)): 
                    chrom[random.randint(1, self.chrom_len-1)] = random.randint(0, self.num_qubits-1)
                    qubitset = set(chrom[1:])
                gene.append(chrom)
            self.population_genes.append(gene)
        #for each generation
        for g in range(gens):
            #evaluate fitness of population
            '''for k, gene in enumerate(self.population_genes):
                program_string = self.program_constructor(gene)
                fitness = self.evaluate_fitness(program_string)
                fitnesses[k] = fitness'''
            
            programs = []
            for k, gene in enumerate(self.population_genes):
                program_string = self.program_constructor(gene)
                programs.append(program_string)
            
            with ThreadPool(32) as p:
                results = p.map(self.evaluate_fitness, list(enumerate(programs)))

            #sort population_genes by fitness
            self.population_genes, self.fitnesses = zip(*sorted(list(zip(self.population_genes, self.fitnesses)), key=itemgetter(1)))
            self.population_genes, self.fitnesses = list(self.population_genes), list(self.fitnesses)

            #print generation stats
            print('Generation ', g+1 , ' top ', n_top, ' architectures: ')
            for gene, fitness in zip(self.population_genes[-n_top:], self.fitnesses[-n_top:]):
                print('Gene: ', gene, ' Fitness: ', fitness)
            print()
            
            with open('Results.txt', 'a+') as f:
                f.write('Gene: '+str(self.population_genes[-1])+' Fitness: '+str(self.fitnesses[-1])+'\n\n')
                
            if self.fitnesses[-1]>self.best_fitness:
                self.best_fitness = self.fitnesses[-1]
                self.best_gene = self.population_genes[-1]
            
            if g!=(gens-1): #no need to breed and mutate on final generation
                #breed to replace least fit members of population
                self.breed(breed_percent=0.9)

                #mutate all members
                self.mutate(mutation_prob=0.1)
        with open('Results.txt', 'a+') as f:
            f.write('Best Gene: '+str(self.best_gene)+' Fitness: '+str(self.best_fitness)+'\n\n')
    
    def test_best(self, epochs=5, lr=0.001):
        '''Constructs the neural network from the best candidate in the gene pool
        and trains it for the given number of epochs before testing its accuracy'''
        nn = self.nn_constructor(self.population_genes[-1])
        fitness = self.evaluate_fitness(nn, epochs=epochs, lr=lr)
        accuracy = self.test_nn(nn)
        return self.population_genes[-1], accuracy
    
    def test_nn(self, nn):
        print("You need to overwrite this function with a function that takes a neural network in as a parameter and returns it's test accuracy")
        assert False

In [8]:
learner = DDQCL(init_depth=10)
#gene1 = [[0, 2, 1, 3], [4, 1, 2, 3], [3, 3, 2, 1]]
#gene2 = [[1, 3, 2, 0], [2, 2, 3, 1], [3, 3, 2, 0]]
'''print(learner.program_constructor(gene1))
print(learner.program_constructor(gene2))

child = learner.crossover(gene1, gene2)
print()
print(learner.program_constructor(child))'''

#learner.run(pop_size=2000, gens=1000)

'print(learner.program_constructor(gene1))\nprint(learner.program_constructor(gene2))\n\nchild = learner.crossover(gene1, gene2)\nprint()\nprint(learner.program_constructor(child))'

In [14]:
best_program = learner.program_constructor(  [[1, 2, 3], [2, 1, 0], [3, 3, 2], [3, 3, 2], [3, 2, 3], [1, 3, 2], [3, 2, 3], [2, 1, 2], [3, 2, 1], [2, 1, 0], [4, 0, 1], [2, 1, 0], [1, 0, 1], [2, 1, 0], [2, 1, 0], [3, 3, 2], [3, 2, 3], [4, 2, 3], [4, 0, 1], [3, 1, 2], [3, 3, 2], [3, 1, 2], [2, 1, 2], [5, 2, 3], [0, 0, 1], [2, 2, 3], [3, 1, 0], [1, 2, 3], [3, 3, 2], [1, 2, 1], [4, 3, 2], [2, 0, 1], [1, 2, 3], [3, 3, 2], [2, 3, 2], [2, 0, 1], [3, 3, 2], [0, 1, 2], [4, 2, 3], [2, 2, 1], [1, 2, 1], [3, 2, 3], [3, 2, 1], [4, 2, 1], [5, 2, 1], [5, 1, 0], [5, 1, 2], [3, 1, 0], [1, 2, 1], [1, 2, 3], [3, 2, 1], [4, 1, 0], [3, 2, 1], [4, 3, 2], [3, 2, 1], [4, 0, 1], [2, 1, 0], [1, 2, 1], [5, 0, 1], [1, 2, 3], [1, 3, 2], [5, 2, 3], [5, 2, 1], [4, 2, 3], [4, 0, 1], [1, 2, 1], [3, 2, 1]]

)
print(best_program)

Y 2 
H 1 
Z 3 
Z 3 
Z 2 
Y 3 
Z 2 
H 1 
Z 2 
H 1 
CNOT 0 1 
H 1 
Y 0 
H 1 
H 1 
Z 3 
Z 2 
CNOT 2 3 
CNOT 0 1 
Z 1 
Z 3 
Z 1 
H 1 
ISWAP 2 3 
X 0 
H 2 
Z 1 
Y 2 
Z 3 
Y 2 
CNOT 3 2 
H 0 
Y 2 
Z 3 
H 3 
H 0 
Z 3 
X 1 
CNOT 2 3 
H 2 
Y 2 
Z 2 
Z 2 
CNOT 2 1 
ISWAP 2 1 
ISWAP 1 0 
ISWAP 1 2 
Z 1 
Y 2 
Y 2 
Z 2 
CNOT 1 0 
Z 2 
CNOT 3 2 
Z 2 
CNOT 0 1 
H 1 
Y 2 
ISWAP 0 1 
Y 2 
Y 3 
ISWAP 2 3 
ISWAP 2 1 
CNOT 2 3 
CNOT 0 1 
Y 2 
Z 2 
MEASURE 0[0] 
MEASURE 1[1] 
MEASURE 2[2] 
MEASURE 3[3]


In [12]:
def execute_alt(quil_program, trials=1000, silent=False, raw=False):
    """
    Thin function that takes a low-level Quil program and returns the
    resulting probability distribution.
    """

    qvm = QVMConnection()
    results = qvm.run(Program(quil_program), trials=trials)
    results = list(map(tuple, results))

    if not silent:
        observed_results = set(results)
        for result in sorted(observed_results):
            bitstring = ''.join(reversed(list(map(str, result))))
            print(f'|{bitstring}> state: {results.count(result)/len(results)} [{results.count(result)}/{len(results)}]')
        if raw:
            print(f'Results: {results}')

In [13]:
execute_alt(best_program)

|0000> state: 0.11 [110/1000]
|1100> state: 0.14 [140/1000]
|1010> state: 0.115 [115/1000]
|0110> state: 0.119 [119/1000]
|1001> state: 0.123 [123/1000]
|0101> state: 0.131 [131/1000]
|0011> state: 0.125 [125/1000]
|1111> state: 0.137 [137/1000]


In [74]:
from pyquil.api import CompilerConnection, get_devices
from pyquil.quil import Pragma, Program
from pyquil.gates import CNOT, H
import ast

def gate_compiler(rig_Code):
    
    devices = get_devices(as_dict=True)
    agave = devices['8Q-Agave']
    compiler = CompilerConnection(agave)

    job_id = compiler.compile_async(Program(learner.program_constructor(ast.literal_eval(rig_Code.strip()))))
    job = compiler.wait_for_job(job_id)
    
    print('compiled quil', job.compiled_quil())
    return (job.gate_depth(),job.program_fidelity())


In [57]:
gate_compiler(codes[1])

job PCCdbekFPCTnSFZv is currently compiling


(66, 0.0)

In [63]:
probsa= np.loadtxt(".\probs.txt")
codes = np.loadtxt(".\codes.txt", dtype=str, delimiter='\n')
infoline = list(map(gate_compiler,codes))
alltogethernow = list(zip(probsa,codes))


job lAsmBJKQcVnBcelP is currently queued for compilation
job BAxCtrKTDYmLYrYd is currently compiling
job PfUFQKxyAOyeehtT is currently compiling
job VQzONBcdoSHKnMfi is currently compiling
job IGoeQTCeaIhvknJP is currently compiling
job LJecgdPnXltBcelO is currently compiling
job mOyJnHZXSltVTBha is currently compiling
job gzWfaOVZowGGSseU is currently compiling
job WhNEKBBKaioVIbbV is currently compiling
job IpEdpmLTUuXYNoGg is currently compiling
job LvbPnDuvHjdDRbgc is currently compiling
job KJRsOgAFRyRtQQfW is currently queued for compilation
job WHlZKBKzwuEklLWX is currently compiling
job nfrDKvyWSaCmKDyr is currently compiling
job GOkGHwZCFqRGUriH is currently compiling
job TxhuNtgHnLTOGExQ is currently compiling
job JwQxWZuQzIkBURMj is currently queued for compilation
job GDYbOwYrHbfesYPJ is currently compiling
job SMZxNUvncqAQBMHz is currently compiling
job utmGVYWhmxwptYte is currently compiling
job wckshvplMHRcyVfC is currently compiling
job VMjZZmnXoPLtVbHz is currently com

In [77]:
baka = list(zip(infoline,probsa))
for i,x in enumerate(baka):
    print(str(i) , x)

0 ((44, 0.0), 0.883006645)
1 ((66, 0.0), 0.882382522)
2 ((84, 0.0), 0.881132181)
3 ((135, 0.0), 0.881132181)
4 ((95, 0.0), 0.881132181)
5 ((77, 0.0), 0.880505961)
6 ((107, 0.0), 0.879879039)
7 ((53, 0.0), 0.878623087)
8 ((36, 0.0), 0.878623087)
9 ((73, 0.0), 0.878623087)
10 ((30, 0.0), 0.877994054)
11 ((52, 0.0), 0.877994054)
12 ((82, 0.0), 0.877994054)
13 ((102, 0.0), 0.877364315)
14 ((60, 0.0), 0.877364315)
15 ((57, 0.0), 0.877364315)
16 ((31, 0.0), 0.876733869)
17 ((61, 0.0), 0.876733869)
18 ((74, 0.0), 0.876733869)
19 ((64, 0.0), 0.876733869)
20 ((44, 0.0), 0.876733869)
21 ((94, 0.0), 0.876733869)
22 ((69, 0.0), 0.876102714)
23 ((64, 0.0), 0.874838274)
24 ((58, 0.0), 0.874838274)
25 ((46, 0.0), 0.874838274)
26 ((89, 0.0), 0.874838274)
27 ((66, 0.0), 0.874838274)
28 ((99, 0.0), 0.874838274)
29 ((113, 0.0), 0.874838274)
30 ((103, 0.0), 0.874838274)
31 ((48, 0.0), 0.874204987)
32 ((75, 0.0), 0.874204987)
33 ((80, 0.0), 0.874204987)
34 ((111, 0.0), 0.874204987)
35 ((76, 0.0), 0.8742049

In [54]:
import ast

ast.literal_eval(codes[1].strip())

[[1, 2, 3],
 [2, 1, 0],
 [3, 3, 2],
 [3, 3, 2],
 [3, 2, 3],
 [1, 3, 2],
 [3, 2, 3],
 [2, 1, 2],
 [3, 2, 1],
 [2, 1, 0],
 [4, 0, 1],
 [2, 1, 0],
 [1, 0, 1],
 [2, 1, 0],
 [2, 1, 0],
 [3, 3, 2],
 [3, 2, 3],
 [4, 2, 3],
 [4, 0, 1],
 [3, 1, 2],
 [3, 3, 2],
 [3, 1, 2],
 [2, 1, 2],
 [5, 2, 3],
 [0, 0, 1],
 [2, 2, 3],
 [3, 1, 0],
 [1, 2, 3],
 [3, 3, 2],
 [1, 2, 1],
 [4, 3, 2],
 [2, 0, 1],
 [1, 2, 3],
 [3, 3, 2],
 [2, 3, 2],
 [2, 0, 1],
 [3, 3, 2],
 [0, 1, 2],
 [4, 2, 3],
 [2, 2, 1],
 [1, 2, 1],
 [3, 2, 3],
 [3, 2, 1],
 [4, 2, 1],
 [5, 2, 1],
 [5, 1, 0],
 [5, 1, 2],
 [3, 1, 0],
 [1, 2, 1],
 [1, 2, 3],
 [3, 2, 1],
 [4, 1, 0],
 [3, 2, 1],
 [4, 3, 2],
 [3, 2, 1],
 [4, 0, 1],
 [2, 1, 0],
 [1, 2, 1],
 [5, 0, 1],
 [1, 2, 3],
 [1, 3, 2],
 [5, 2, 3],
 [5, 2, 1],
 [4, 2, 3],
 [4, 0, 1],
 [1, 2, 1],
 [3, 2, 1]]

In [52]:
code[1].strip()

NameError: name 'code' is not defined

In [79]:
print(learner.program_constructor(ast.literal_eval(codes[63].strip())))

Y 3 
H 0 
Y 2 
H 2 
X 2 
CNOT 2 1 
Y 3 
H 3 
ISWAP 3 2 
H 2 
X 1 
X 3 
H 2 
H 2 
Z 1 
Y 2 
CNOT 3 2 
H 3 
H 3 
CNOT 2 3 
Y 3 
Y 1 
CNOT 0 1 
MEASURE 0[0] 
MEASURE 1[1] 
MEASURE 2[2] 
MEASURE 3[3]


In [78]:
gate_compiler(codes[63])

job cXqchSQjdzaLhvas is currently queued for compilation
compiled quil PRAGMA EXPECTED_REWIRING "#(0 1 2 3 4 5 6 7)"
RZ(pi/2) 0
RX(pi/2) 0
RZ(pi/2) 0
RZ(-pi/2) 1
RX(pi/2) 1
RZ(pi/2) 2
RX(1.5707963267948968) 2
CZ 1 2
CZ 1 0
RZ(-1.5707963267948997) 2
RX(pi/2) 2
RZ(-1.8103498726943617) 2
RX(-pi/2) 2
RX(pi/2) 3
RZ(2.141636944514338) 3
RX(-pi/2) 3
CZ 2 3
RX(pi/2) 2
RZ(-3.1415926535897776) 3
RX(-1.5707963267948948) 3
CZ 2 3
RZ(pi) 0
RX(-pi/2) 1
RZ(pi/2) 1
RX(-pi/2) 2
RZ(0.9999557090754599) 2
RX(pi/2) 3
RZ(1.8103498726943632) 3
RX(-pi/2) 3
RZ(-1.5707963267948986) 3
PRAGMA CURRENT_REWIRING "#(0 1 2 3 4 5 6 7)"
PRAGMA EXPECTED_REWIRING "#(0 1 2 3 4 5 6 7)"
MEASURE 0 [0]
MEASURE 1 [1]
MEASURE 2 [2]
MEASURE 3 [3]
PRAGMA CURRENT_REWIRING "#(0 1 2 3 4 5 6 7)"



(15, 0.49327830327297933)

In [80]:
execute_alt("""Y 3 
H 0 
Y 2 
H 2 
X 2 
CNOT 2 1 
Y 3 
H 3 
ISWAP 3 2 
H 2 
X 1 
X 3 
H 2 
H 2 
Z 1 
Y 2 
CNOT 3 2 
H 3 
H 3 
CNOT 2 3 
Y 3 
Y 1 
CNOT 0 1 
MEASURE 0[0] 
MEASURE 1[1] 
MEASURE 2[2] 
MEASURE 3[3]""")

|0000> state: 0.121 [121/1000]
|1100> state: 0.132 [132/1000]
|1010> state: 0.114 [114/1000]
|0110> state: 0.121 [121/1000]
|1001> state: 0.142 [142/1000]
|0101> state: 0.118 [118/1000]
|0011> state: 0.114 [114/1000]
|1111> state: 0.138 [138/1000]
